In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
import matplotlib.pyplot as plt




In [ ]:
!unzip smileys.zip

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [ ]:
# class Generator(nn.Module):
#     def __init__(self, noise_dim=100, channels=3):
#         super(Generator, self).__init__()
#         self.noise_dim = noise_dim
#         self.channels = channels
#         channels = 256

#         self.main = nn.Sequential(
#             nn.ConvTranspose2d(self.noise_dim, channels * 8, 4, 1, 0, bias=False),
#             nn.BatchNorm2d(channels * 8),
#             nn.ReLU(True),

#             nn.ConvTranspose2d(channels * 8, channels * 4, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(channels * 4),
#             nn.ReLU(True),

#             nn.ConvTranspose2d(channels * 4, channels * 2, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(channels * 2),
#             nn.ReLU(True),

#             nn.ConvTranspose2d(channels * 2, channels, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(channels),
#             nn.ReLU(True),

#             nn.ConvTranspose2d(channels, self.channels, 4, 2, 1, bias=False),
#             nn.Tanh()
#         )

#     def forward(self, input):
#         return self.main(input)


class Generator(nn.Module):
    def __init__(self, noise_dim=100, channels=3, channels=512):
        super(Generator, self).__init__()
        self.noise_dim = noise_dim
        self.channels = channels

        self.main = nn.Sequential(
            nn.ConvTranspose2d(self.noise_dim, channels * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(channels * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(channels * 8, channels * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(channels * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(channels * 4, channels * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(channels * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(channels * 2, channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(channels),
            nn.ReLU(True),

            nn.ConvTranspose2d(channels, self.channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:

class Discriminator(nn.Module):
    def __init__(self, channels=3, d_channels=32):
        super(Discriminator, self).__init__()
        self.channels = channels

        self.main = nn.Sequential(
            nn.Conv2d(self.channels, d_channels, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(d_channels, d_channels * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(d_channels * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(d_channels * 2, d_channels * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(d_channels * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(d_channels * 4, d_channels * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(d_channels * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(d_channels * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

dataroot = "/kaggle/input/uploads/smileys"
workers = 2
batch_size = 8
image_size = 64
noise_dim = 100
channels = 512
d_channels = 32
num_epochs = 500
lr = 0.00015
beta1 = 0.4

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = ImageFolder(root=dataroot, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

print(f"Dataset size: {len(dataset)} images")
print(f"Number of batches per epoch: {len(dataloader)}")
print(f"Training parameters: nc={nc}, noise_dim={noise_dim}, 
Generator channels={channels}, d_channels={d_channels}")

generator = Generator(noise_dim, 3, channels).to(device)
generator.apply(weights_init)

discriminator = Discriminator(3, d_channels).to(device)
discriminator.apply(weights_init)

criterion = nn.BCELoss()

fixed_noise = torch.randn(64, noise_dim, 1, 1, device=device)
real_label = 1
fake_label = 0

optimizerD = optim.Adam(discriminator.parameters(), lr=lr * 0.7, betas=(beta1, 0.999))
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))

G_losses = []
D_losses = []

print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        discriminator.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        output = discriminator(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, noise_dim, 1, 1, device=device)
        fake = generator(noise)
        label.fill_(fake_label)
        output = discriminator(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake

        if errD.item() > 0.1:
            optimizerD.step()

        generator.zero_grad()
        label.fill_(real_label)
        output = discriminator(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        if i % 5 == 0:
            print(f'[{epoch + 1}/{num_epochs}][{i}/{len(dataloader)}]\tLoss_D: {errD.item():.4f}\tLoss_G: {errG.item():.4f}\tD(x): {D_x:.4f}\tD(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f}')

        G_losses.append(errG.item())
        D_losses.append(errD.item())

        iters += 1

    if epoch % 100 == 0:
        torch.save(generator.state_dict(), f'generator_epoch{epoch}.pth')
        torch.save(discriminator.state_dict(), f'discriminator_epoch{epoch}.pth')

torch.save(generator.state_dict(), 'generator_final.pth')
torch.save(discriminator.state_dict(), 'discriminator_final.pth')

plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig('training_loss.png')
plt.close()

Using device: cuda:0
Dataset size: 108 images
Number of batches per epoch: 7
Training parameters: nc=3, nz=100, ngf=512, ndf=32
Starting Training Loop...
[1/500][0/7]	Loss_D: 1.0262	Loss_G: 1.6620	D(x): 0.6297	D(G(z)): 0.3995 / 0.2141
[1/500][5/7]	Loss_D: 4.9284	Loss_G: 0.2016	D(x): 0.9603	D(G(z)): 0.9870 / 0.8279
[2/500][0/7]	Loss_D: 5.7789	Loss_G: 0.0825	D(x): 0.9331	D(G(z)): 0.9956 / 0.9223
[2/500][5/7]	Loss_D: 4.4373	Loss_G: 0.6171	D(x): 0.7745	D(G(z)): 0.9736 / 0.5753
[3/500][0/7]	Loss_D: 4.8397	Loss_G: 0.4930	D(x): 0.8677	D(G(z)): 0.9865 / 0.6352
[3/500][5/7]	Loss_D: 3.0167	Loss_G: 1.9080	D(x): 0.8791	D(G(z)): 0.9303 / 0.1673
[4/500][0/7]	Loss_D: 2.5754	Loss_G: 2.4932	D(x): 0.8588	D(G(z)): 0.8864 / 0.0940
[4/500][5/7]	Loss_D: 2.0177	Loss_G: 2.9144	D(x): 0.8746	D(G(z)): 0.8266 / 0.0612
[5/500][0/7]	Loss_D: 1.6968	Loss_G: 3.0428	D(x): 0.7878	D(G(z)): 0.7005 / 0.0642
[5/500][5/7]	Loss_D: 1.3288	Loss_G: 3.4263	D(x): 0.7882	D(G(z)): 0.6178 / 0.0356
[6/500][0/7]	Loss_D: 1.2030	Loss_G: 

In [19]:
!rm generator_epo*

In [20]:
!rm discriminator_epo*

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [5]:
generator = Generator().to(device)
generator.load_state_dict(torch.load('/kaggle/input/uploads/generator_final.pth'))
generator.eval()

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 4096, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(4096, 2048, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(2048, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(512, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): T

In [11]:
# noise = torch.randn(1, 100, 1, 1, device=device)

# fake = generator(noise)


Generations

In [ ]:
noise = torch.randn(1, 100, 1, 1, device=device)

fake = generator(noise)

plt.figure(figsize=(3,3))
plt.imshow(fake.detach().cpu()[0].permute(1,2,0).numpy())
plt.axis('off')
plt.show()